In [1]:
!pip install wordcloud

  Obtaining dependency information for wordcloud from https://files.pythonhosted.org/packages/f5/b0/247159f61c5d5d6647171bef84430b7efad4db504f0229674024f3a4f7f2/wordcloud-1.9.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/300.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.2 kB ? eta -:--:--
   ----- --------------------------------- 41.0/300.2 kB 653.6 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/300.2 kB 744.7 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/300.2 kB 717.5 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/300.2 kB 717.5 kB/s eta 0:00:01
   ---------------------------------------  297.0/300.2 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 300.2/300.2 kB 1.1 MB/s eta 0:00:00


In [2]:
pip install pymongo

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/c8/1a/1ba5578cc3acc8ef0dfd6eb9385f9c4b0fe3039665af93bcf062c34fcdb6/pymongo-4.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
   -- ------------------------------------- 30.7/472.7 kB ? eta -:--:--
   --- ----------------------------------- 41.0/472.7 kB 653.6 kB/s eta 0:00:01
   --- ----------------------------------- 41.0/472.7 kB 653.6 kB/s eta 0:00:01
   ----- --------------------------------- 71.7/472.7 kB 393.8 kB/s eta 0:00:02
   ----- --------------------------------- 71.7/472.7 kB 393.8 kB/s eta 0:00:02
   ------- -----------------------------

In [2]:
from wordcloud import WordCloud
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
# !pip install neattext
from keras.models import load_model
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [3]:
from keras.utils import pad_sequences
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [4]:
data=pd.read_csv('new_data.txt')
data.head(20)

,Project Idea,Brainstorming Idea
0,An app for organizing outdoor team-building ac...,Mind Mapping
1,A website for DIY party decoration ideas,Random Word Association
2,A platform for virtual reality-based party exp...,Reverse Thinking
3,An AI-powered party planning assistant,Mind Mapping
4,A social media platform for sharing party play...,Random Word Association
5,A service for renting themed party decorations,Reverse Thinking
6,An app that generates party game ideas,Mind Mapping
7,A marketplace for custom-designed party invita...,Random Word Association
8,A platform for hosting virtual reality parties,Reverse Thinking
9,An online community for sharing party planning...,Mind Mapping


In [5]:
data['Brainstorming Idea'].value_counts()

Brainstorming Idea
Mind Mapping               427
Random Word Association     13
Reverse Thinking            12
Name: count, dtype: int64

In [6]:
data['Brainstorming Idea'].value_counts().index.values

array(['Mind Mapping', 'Random Word Association', 'Reverse Thinking'],
      dtype=object)

In [7]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [8]:
train_data['Brainstorming Idea'].value_counts().index.values

array(['Mind Mapping', 'Reverse Thinking', 'Random Word Association'],
      dtype=object)

In [9]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [10]:
cleaned_train_text,train_text_length=clean_text(train_data["Project Idea"])
cleaned_test_text,test_text_length=clean_text(test_data["Project Idea"])

100%|██████████| 91/91 [00:00<?, ?it/s]


In [11]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [12]:
word_freq=pd.DataFrame(tokenizer.word_counts.items(),columns=['word','count']).sort_values(by='count',ascending=False)

In [13]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=100)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=100)

In [14]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['Brainstorming Idea'])
test_output=lbl_target.transform(test_data['Brainstorming Idea'])

In [15]:
with open('glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [16]:
v=len(tokenizer.word_index)
embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [17]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

In [26]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, GlobalMaxPooling1D, Dense

# Assuming you have 'v' defined as the vocabulary size and 'embedding_matrix' as the pre-trained embedding matrix

# Define the model
model = keras.Sequential([
    keras.layers.Input(shape=(100,)),
    keras.layers.Embedding(v + 1, 300, weights=[embedding_matrix], trainable=False),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # Output layer with 3 units for 3 categories
])

# Compile the model
model.compile(optimizer=keras.optimizers.SGD(0.05, momentum=0.09),
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for multi-class classification
              metrics=['accuracy'])


ChatCompletionMessage(content=' Person A: Create the visually appealing designs for the website.\nPerson B: Code the functionalities for the website.\nPerson C: Manage the sales transactions for the website.\nPerson D: Handle customer inquiries and support on the website.\nPerson E: Coordinate with the team for any updates or changes to the website.\n', role='assistant', function_call=None, tool_calls=None)


In [27]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=64,callbacks=[early_stop,reducelr],)

Epoch 1/20
4/6 [===================>..........] - ETA: 0s - loss: 0.9818 - accuracy: 0.7266 

InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\HP\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\HP\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\HP\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\HP\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\HP\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\HP\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\HP\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\HP\anaconda3\Lib\site-packages\ipyk

In [20]:
suicide_model.save('my_model.h5')

In [35]:
suicide_model = load_model('my_model.h5')

In [36]:
def detectSuicide(text):
    cleaned_text, length = clean_text(text)
    tokens = tokenizer.texts_to_sequences(cleaned_text)
    paded = pad_sequences(tokens,maxlen=50)
    if suicide_model.predict([paded]) >=0.50:
#         print("contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 0
    else:
#         print("Doesn't contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 1
detectSuicide(["I don't want to live anymore"])

100%|██████████| 1/1 [00:00<00:00, 1004.86it/s]


1/1 [==============================] - 0s 425ms/step


0

In [37]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [39]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Transformer(nn.Module):
    def __init__(self, num_classes):
        super(Transformer, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output[1]
        logits = self.fc(pooled_output)
        return logits

In [40]:
model = Transformer(num_classes=7)
model.load_state_dict(torch.load("model.pth", map_location= torch.device("cpu")))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [41]:
text = "The only reason I haven’t killed myself yet is because of my mom and brothers. If I didn’t have them I’d be dead long ago. "
tokens = bert_tokenizer.encode_plus(
    text,
    max_length=512,
    
    add_special_tokens=True,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
)
output = model((tokens['input_ids']), (tokens['attention_mask']))
predicted_label = torch.argmax(output, dim=1).item()
print(predicted_label)

0


In [42]:
detectSuicide(["i do not want to live anymore"])

100%|██████████| 1/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 25ms/step


0

In [43]:
i need help just help me im crying so hard. There is no point living now
I am miserable .My life is over now.
The only reason I haven’t killed myself yet is because of my mom and brothers.If I didn’t have them I’d be dead

SyntaxError: invalid syntax (2067087818.py, line 1)

In [ ]:
suicide_model.save("model_2.h5")

In [ ]:
data["text"][20]

In [ ]:
list= ["i need helpjust help me im crying so hard", "Oh Fuck! can't take this anymore.I am tired of this.", "This is it , I can't live anymore.", "I am gonna hurt myself",
      "I do not want to self harm again", "It ends tonight.I can’t do it anymore.I quit.", "I am endng it .This ends now.",
      "what would be a painless method of suicide","I want to endmy life", "I want to commit suicide"]

In [ ]:
from flask import Flask
app = Flask(__name__)
@app.route('/')
def hello_world():
    return 'Hello, World!'
if __name__ == '__main__':
    app.run()

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
app = Flask(__name__)
from pymongo import MongoClient
import json

client = MongoClient('mongodb://localhost:27017')
db = client.serene
collection = db.users

@app.route('/reg', methods=['POST'])
def receive_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()  # Get the JSON data sent from the client
    # Process the data
    #

    result = collection.insert_one(data)
    res= {"data" : "hello"}
    return jsonify(res)
@app.route('/model', methods=['POST'])
def model_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id = data['id']
    id = str(id)
    mail = data['mail']
    print(id)
    
    df = pd.read_csv('pulse.csv')
    dff=df['rate']

    m = detectSuicide([id])   
    for i in dff:
        up_doc = collection.find_one_and_update(
            {'Email': mail},
            {'$push': {'Pulse': i}}
        )
        
    print(m)
    if m==1:
        text = id
        print(text)
        tokens = bert_tokenizer.encode_plus(
                        text,
                        max_length=512,
    
                        add_special_tokens=True,
                        return_token_type_ids=False,
                        padding='max_length',
                        return_attention_mask=True,
                        return_tensors='pt',
                    )
        output = model((tokens['input_ids']), (tokens['attention_mask']))
        m = int(torch.argmax(output, dim=1).item())
        m=m+1
    
    print(m)
    print(mail)
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB
    updated_doc = collection.find_one_and_update(
        {'Email': mail},
        {'$push': {'Mood': m}}
    )

    lst = collection.find_one({'Email': mail })
    if lst is not None:
        # extract mylist array from document
#         mylist = lst['Mood']
        json_data = jsonify(m)
        return json_data

    # res = {"data": "hello"}
    # return jsonify(id)

    else:
        # return error response if document not found
        return jsonify({'error': 'Document not found'})

    # Send a JSON response back to the client
# Press the green button in the gutter to run the script.

@app.route('/login', methods=['POST'])
def login_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id = data['id']
    pas = data['pass']
    
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB.
    lst = collection.find_one({'Email': id})
    if lst is not None:
        # extract mylist array from document
        checkpass = lst['Password']
        if(str(pas)==checkpass):
            return jsonify(id)
        else:
            code={'error' : 'existing user'}
            return jsonify(code)
    else:
        cod = {'error' : 'User doest exists'}
        return  jsonify((cod))
        # json_data = jsonify(mylist)
        # return json_data

@app.route('/graph', methods=['POST'])
def chart_data():
    # data = {
    #     'Name': request.form.get('Name'),
    #     'Email': request.form.get('Email'),
    #     'Number': request.form.get('Number'),
    #     'Password': request.form.get('Password')
    # }
    data = request.get_json()
    id1 = data
    
    # Get the JSON data sent from the client
    # Process the data
    # insert value into list in MongoDB.
    lst = collection.find_one({'Email': id1})
    if lst is not None:
        # extract mylist array from document
        md = lst['Mood']
        return jsonify(md)
    else:
        cod = {'error' : 'User doest exists'}
        return  jsonify((cod))        
        
        
@app.route('/')
def hello_world():
    return 'Hello, World!, wecome to the home page'


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jan/2024 22:49:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2024 22:49:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2024 22:49:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2024 22:50:02] "GET / HTTP/1.1" 200 -


In [33]:
pip install deepface

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [34]:
app = Flask(__name__)
app.config['MONGO_URI'] = 'mongodb://localhost:27017/serene'  # MongoDB URI


In [40]:
from flask import Flask, jsonify
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError

app = Flask(__name__)
app.config['MONGO_URI'] = 'mongodb://localhost:27017/serene'

client = None
try:
    client = MongoClient(app.config['MONGO_URI'], serverSelectionTimeoutMS=5000)
    client.server_info()   # Test the connection
    db = client.get_database()
    collection = db.get_collection('sangs')
    print("Connected to MongoDB")
except ServerSelectionTimeoutError:
    print("Failed to connect to MongoDB")


Connected to MongoDB
